# Integer programming

With integer programming, we now allow discrete values in our linear programs.

This can be useful for including:
- Set-up costs (i.e. the first item is expensive, but then each item is a subsequent linear cost).
- Facility location
- Machine scheduling

Broadly speaking, it's useful for selection and assignment.

## More specific definitions

If we formulate and solve a model with integer variables, it's Integer Programming.

This is typically a linear IP (LIP)

But if we have a nonlinear objective function or a functional constraint, it's a nonlinear (NLIP).


## An example: The knapsack problem, revisited

Say we have four items with values and weights:

* Item 1, $16 and 5kg
* Item 2, $22 and 7kg
* Item 3, $12 and 4kg
* Item 4, $8 and 3kg

The knapsack capacity is 10kg.

In [12]:
import pulp

In [9]:
prob = pulp.LpProblem("Knapsack", pulp.LpMaximize)

item1 = pulp.LpVariable("Item1", 0, 1, pulp.LpInteger)
item2 = pulp.LpVariable("Item2", 0, 1, pulp.LpInteger)
item3 = pulp.LpVariable("Item3", 0, 1, pulp.LpInteger)
item4 = pulp.LpVariable("Item4", 0, 1, pulp.LpInteger)

prob += (item1*16 + item2*22 + item3*12 + item4*8, "ValueMaximisation")
prob += (item1*5 + item2*7 + item3*4 + item4*3 <= 10.0, "WeightLimit")

In [10]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Goyder/.local/share/virtualenvs/ops-research-fbgn6z6J/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/g2/6w3bm3ts67nbv8gqz10hmcbh0000gn/T/4c16615f7296460b9286dc99a33350bd-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/g2/6w3bm3ts67nbv8gqz10hmcbh0000gn/T/4c16615f7296460b9286dc99a33350bd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 23 RHS
At line 25 BOUNDS
At line 30 ENDATA
Problem MODEL has 1 rows, 4 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 31.7143 - 0.00 seconds
Cgl0004I processed model has 1 rows, 4 columns (4 integer (4 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.285714
Cbc0038I Pass   1

1

In [11]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Item1 = 0.0
Item2 = 1.0
Item3 = 0.0
Item4 = 1.0


## Integer programming formulation

The use of integers allows us to define a lot of interesting constraints.

Say we have a set of binary variables, $x_i, \{0, 1\}, \forall_i = 1,\cdots, 4$.

If we must select at least one item among items 2, 3, and 4: $$x_2 + x_3 + x_4 \ge 1$$

If we must select at most two items among items 1, 3, and 4: $$x_1 + x_3 + x_4 \le 2$$

If we must select item 2 or item 3: $$x_2 + x_3 \ge 1$$

Select item 2, otherwise select items 3 and 4 together: $$2x_2 + x_3 + x_4 \ge 2$$

If item 2 is selected, select item 3: $$x_2 \le x_3$$

If item 1 is selected, do not select items 3 and 4: $$2(1-x_1) \ge x_3 + x_4$$

**At least/most some constraints**

Suppose we want to satisfy one of the two constraints, $g_1(x)\le b_1$ and $g_2(x) \le b_2$.

We can define binary variables:

$$
z_1 = 1 \text{ if constraint 1 is satisfied or 0 otherwise}\\
z_2 = 1 \text{ if constraint 2 is satisfied or 0 otherwise}
$$

With $M_i$ being an upper bound of each LHS, the following two constraints implement what we need:

$$
g_1(x) - b_1 \le M(1-z_1)\\
g_2(x) - b_2 \le M(1-z_2)\\
z_1 + z_2 \ge 1\\
z_1, z_2 \in \{0, 1\}
$$

## Fixed-charge constraints, or set-up costs

Consider the setup cost at factory $i$, $S_i$. You need to pay this setup cost as long as any positive amount of products is produced.

As a first run, let the decision variables be 

$$x_i = \text{Production quantity at factory } i, i=1, \ldots, n,\\
y_i = \begin{cases}
1 \text{\quad if some productions are produced at factory }i, i=1,\ldots,n,\\
0 \text{\quad o/w.}
\end{cases}$$

And the objective function:

$$min \sum_{i=1}^n C_i x_i + \sum_{i=1}^n S_i y_i $$

And we have some kind of capacity and demand constraints:
$$ x_i \le K_i \forall i = 1,\ldots,n \\
\sum_{i=1}^n x_i \ge D$$

And to factor in the setup costs, we modify the capacity:
$$x_i \le K_iy_i \quad \forall i=1,\ldots,n$$

(This creates our general "fixed charge constraint" – $x_i \le K_i y_i$, where K_i is the upper bound.)

And our binary and nonnegative constraints:

$$x_i \ge 0\\ y \in \{0, 1\} \quad \forall i=1, \ldots,n$$

## Extending setup costs to an application: Minimum covering problems

Consider a set of demands, $I$, and a set of locations, $J$.

The distance between demand $i$ and location $j$ is $d_ij \gt 0, i \isin I, j \isin J$.

A service level $s>0$ is given; demand $i$ is "covered" by location $j$ if $d_{ij} \lt s$.

How do you allocate as few facilities as possible to cover all demands?

We define a new parameter, $a_{ij} = 1$ if $d_{ij} < s$, or $0$ otherwise. $i\isin I, j \isin J$.

$x_j=1$ if a facility is built at location $j \isin J$, or $0$ otherwise.

Therefore our formulation becomes 

$$min \quad \sum_{j\isin J} x_j \\
s.t. \quad \sum_{j\isin J} a_{ij}x_j \ge 1\quad \forall i \isin I \\
\quad x_j \isin \{0, 1 \}\quad \forall j \isin J$$

## Fixed charge location problems

* Consider a set of demands, $I$, and a set of locations, $J$.
* At demand $i$, the demand size is $h_i > 0$.
* The unit shipping cost from location $j$ to demand $i$ is $d_{ij} > 0$.
* The fixed construction cost at location j is $f_j > 0$.

Goal: how do we allocate facilities to minimise the total shippping and construction costs?

So we minimise the setup cost and the travel costs...

$$min \sum_{j \isin J} f_j x_j + \sum_{j \isin J} \sum_{i \isin I} h_i y_{ij} d_{ij}$$

Factoring in our setup constraints...

$$ y_ij \le x_j \forall i \isin I, j \isin J \\

\sum_{j \isin J} y_{ij} = 1, \forall{i} \isin I \\

x_j \isin \{0, 1\} \quad \forall j \isin J \\

y_i \isin \{0, 1\} \quad \forall i \isin I
$$

In essence, we assign each demand to a location via the $y_i$ function.

## Machine scheduling

### Minimising single-machine total completion time

*NOTE: the lecturer notes that this is something of a terrible example, because you can't minimise single-machine total completion time – no matter how you order it, they're all going to complete at the same time.*

Consider scheduling $n$ jobs on a single machine. 

Each job, $j \isin J = \{ 1, 2, \ldots, n\}$ has a processing time, $p_j$.

Different schedules give these jobs different completion times, $x_j$.

The machine can only process one job at a time. We want to schedule all the jobs to minimise the total completion time, $\sum_{j \isin J} x_j $.

Our formulation ends up being:

$$
min\quad \sum_{j \isin J} x_j \\
s.t. \quad x_i + p_j - x_j \le Mz_{ij}\quad \forall i \isin J, j \isin J, i < J \\
\quad x_i + p_i - x_i \le M(1-z_{ij})\quad \forall i \isin J, j \isin J, i < J \\
\quad x_j \ge p_j \quad \forall j \isin J \\
\quad x_j \ge 0 \quad \forall j \isin J \\
z_ij \isin \{ 0, 1 \} \quad \forall i \isin J, j \isin J, i < j $$

### Minimising makespan on parallel machines

If we have $n$ jobs on $m$ parallel machines, how do we assign them to minimise the makespan (the total time between start and finish)?

We need to assign jobs to machines. Because there is no ordering component, the sequence does not matter.

Let $x_{ij} = 1$ if job $j \isin J$ is assigned to machine $i \isin I$, $0$ otherwise.

For machine $i \isin I$, the last job is completed at $\sum_j{j \isin J} p_j x_{ij}$.

The makespan $w$ is the maximum completion time among all machines. Therefore, $$w \ge \sum_{j \isin J} p_j x_{ij} \quad \forall i \isin I$$.

## Worked application – Facility location

Say we want to determine the optimal placement of facilities in order to satisfy demands. Our parameters are:

$$
f_j = \text{weekly operating cost of distribution center } j\\
c_{ij} = \text{shipping cost per book from distribution center } j \text{ to region } i\\
K_j = \text{capacity of distribution center }j\\
D_{i} = \text{book demand of region }i. $$

The decision variables are 

$$
x_j = \begin{cases}
1 \text{\quad if a distribution center is built at location }j, i=1,\ldots,n,\\
0 \text{\quad o/w.}
\end{cases} \\
y_{ij} = \text{number of books shipped from distribution center }j \text{ to region }i$$

Our formulation becomes:

$$
min\quad\sum_{j\isin J}f_jx_j + \sum_{j\isin J} \sum_{i\isin I} c_{ij}y_{ij}\\

\begin{align*}
s.t.\quad\sum_{i\isin I}y_{ij} &\le K_jx_j \quad \forall j = 1, \ldots, 5\\
\sum_{j \isin J} y_{ij} &\ge D_i \quad \forall i = 1, \ldots, 5\\
y_{ij} &\ge 0\\
x_j & \isin \{0, 1\} \\

\end{align*}

$$

We'll solve this using our integer programming package. This is one where we'll really have to programmatically define our inputs.

In [67]:
I = 5
J = 5

D_i = {
    0: 8000,
    1: 12000,
    2: 9000,
    3: 14000,
    4: 17000
}

f_j = {
    0: 40000,
    1: 30000,
    2: 25000,
    3: 40000,
    4: 30000
}

K_j = {
    0: 20000,
    1: 20000,
    2: 15000,
    3: 25000,
    4: 15000
}

c_ij = {
    0: {
        0: 2.4,
        1: 3.25,
        2: 4.05,
        3: 5.25,
        4: 6.95
    },
    1: {
        0: 3.5,
        1: 2.3,
        2: 3.25,
        3: 6.05,
        4: 5.85
    },
    2: {
        0: 4.8,
        1: 3.4,
        2: 2.85,
        3: 4.3,
        4: 4.8
    },
    3: {
        0: 6.8,
        1: 5.25,
        2: 4.3,
        3: 3.25,
        4: 2.1 
    },
    4: {
        0: 5.75,
        1: 6,
        2: 4.75,
        3: 2.75,
        4: 3.5
    },
}

In [68]:
prob = pulp.LpProblem("Facility_location", pulp.LpMinimize)

x_j = { j: pulp.LpVariable("x_{}".format(j), 0, 1, pulp.LpInteger) for j in range(J)}
y_ij = { i: { j: pulp.LpVariable("y_{}{}".format(i, j), 0, None, pulp.LpVariable) for j in range(J) } for i in range(I)}

prob += pulp.lpSum(
    [x_j[j] * f_j[j] for j in range(J)] + 
    [y_ij[i][j] * c_ij[i][j] for i in range(I) for j in range(J)]
), "Total cost of operations"

for j in range(J):
    print(j)
    print(sum([y_ij[i][j] for i in range(I)]))
    prob += sum([y_ij[i][j] for i in range(I)]) <= x_j[j] * K_j[j], "Facility production constraint - {}".format(j)

for i in range(I): 
    print(i)
    prob += sum([y_ij[i][j] for j in range(J)]) >= D_i[i], "Demand satisfaction constraint - {}".format(i)

0
y_00 + y_10 + y_20 + y_30 + y_40
1
y_01 + y_11 + y_21 + y_31 + y_41
2
y_02 + y_12 + y_22 + y_32 + y_42
3
y_03 + y_13 + y_23 + y_33 + y_43
4
y_04 + y_14 + y_24 + y_34 + y_44
0
1
2
3
4


In [69]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Goyder/.local/share/virtualenvs/ops-research-fbgn6z6J/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/g2/6w3bm3ts67nbv8gqz10hmcbh0000gn/T/6329c21937c248adb8d5ee568c869a68-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/g2/6w3bm3ts67nbv8gqz10hmcbh0000gn/T/6329c21937c248adb8d5ee568c869a68-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 111 RHS
At line 122 BOUNDS
At line 128 ENDATA
Problem MODEL has 10 rows, 30 columns and 55 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 252800 - 0.00 seconds
Cgl0004I processed model has 10 rows, 30 columns (5 integer (5 of which binary)) and 55 elements
Cbc0038I Initial state - 5 integers unsatisfied sum - 1.58667
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 296400 iterati

1

In [70]:
pulp.value(prob.objective)

268950.0

(This is the optimal solution!)

In [71]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

x_0 = 0.0
x_1 = 1.0
x_2 = 0.0
x_3 = 1.0
x_4 = 1.0
y_00 = 0.0
y_01 = 8000.0
y_02 = 0.0
y_03 = 0.0
y_04 = 0.0
y_10 = 0.0
y_11 = 12000.0
y_12 = 0.0
y_13 = 0.0
y_14 = 0.0
y_20 = 0.0
y_21 = 0.0
y_22 = 0.0
y_23 = 8000.0
y_24 = 1000.0
y_30 = 0.0
y_31 = 0.0
y_32 = 0.0
y_33 = 0.0
y_34 = 14000.0
y_40 = 0.0
y_41 = 0.0
y_42 = 0.0
y_43 = 17000.0
y_44 = 0.0
